In [1]:
!wget https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
!unzip trainDevTestTrees_PTB.zip

--2020-05-13 12:37:56--  https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789539 (771K) [application/zip]
Saving to: ‘trainDevTestTrees_PTB.zip’

trainDevTestTrees_P 100%[===================>] 771.03K   753KB/s    in 1.0s    

2020-05-13 12:37:58 (753 KB/s) - ‘trainDevTestTrees_PTB.zip’ saved [789539/789539]

Archive:  trainDevTestTrees_PTB.zip
   creating: trees/
  inflating: trees/dev.txt           
  inflating: trees/test.txt          
  inflating: trees/train.txt         


In [2]:
import nltk
a = nltk.corpus.BracketParseCorpusReader("trees", "(train|dev|test)\.txt")

text = {}
labels = {}
keys = ['train', 'dev', 'test']
for k in keys :
    text[k] = [x.leaves() for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    labels[k] = [int(x.label()) for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    print(len(text[k]))
    
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
import re

def tokenize(text) :
    text = " ".join(text)
    text = text.replace("-LRB-", '')
    text = text.replace("-RRB-", " ")
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    tokens = " ".join([t.text.lower() for t in nlp(text)])
    return tokens

for k in keys :
    text[k] = [tokenize(t) for t in text[k]]
    labels[k] = [1 if x >= 3 else 0 for x in labels[k]]

6920
872
1821


In [3]:
import pandas as pd
df_texts = []
df_labels = []
df_exp_split = []

for k in keys :
    df_texts += text[k]
    df_labels += labels[k]
    df_exp_split += [k]*len(text[k])
    
data = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_split}) 

In [5]:
import json
from tqdm import tqdm

import os
os.makedirs('data/', exist_ok=True)
for key in ['train', 'dev', 'test'] :
    data_key = data[data.exp_split == key].to_dict(orient='records')
    data_key = [{'document' : x['text'], 'label' : x['label']} for x in tqdm(data_key)]
    f = open('data/' + key + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in data_key]))
    f.close()


100%|██████████| 6920/6920 [00:00<00:00, 1552448.85it/s]

100%|██████████| 872/872 [00:00<00:00, 1859396.59it/s]

100%|██████████| 1821/1821 [00:00<00:00, 1796713.15it/s]


In [6]:
import json
for key in ['train', 'dev', 'test'] :
    data_key = [json.loads(line) for line in open('data/' + key + '.jsonl')]
    for i, d in enumerate(data_key) :
        d['annotation_id'] = key + '_' + str(i)
    f = open('data/' + key + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in data_key]))
    f.close()